In [1]:
from os.path import isfile
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.models import resnet34
import pickle
from sklearn.model_selection import train_test_split
from helper_functions import train_or_load, create_eval_post_loader, create_post_train_loader, evaluate_attack_model, DatasetClassN

Parameters

In [2]:
## Datasets
SHADOW_DATA_PATH = "pickle/cifar10/resnet34/shadow.p"
EVALUATE_DATA_PATH = "pickle/cifar10/resnet34/eval.p"
# Save Dset create for attack model training
ATT_TRAIN_DATA_PATH = "pickle/cifar10/resnet34/attack_train.p"
## Models
SHADOW_MODEL_PATH = "shadow_models/resnet34_shadow_cifar_overtrained.pth"
TARGET_MODEL_PATH = "models/resnet34_cifar10.pth"
ATTACK_MODEL_PATH = "attack_models/attack_resnet_cifar.pth"
## Attack model training parameters
LEARNING_R = 0.0001
EPOCHS = 5
MULTI_WORKERS_N = 2

DEVICE=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Shadow Model

Shadow Model

In [3]:
#load the shadow model trained in the other python script
resnet_shadow = resnet34(weights=None,num_classes=DatasetClassN.Cifar.value).to(DEVICE) #resnet_target is the shadow model
resnet_shadow.load_state_dict(torch.load(SHADOW_MODEL_PATH, map_location=DEVICE))

<All keys matched successfully>

Shadow dataset

In [4]:
with open(SHADOW_DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
    
train_data, val_data = train_test_split(dataset, test_size=(1-0.5),shuffle=False)
  
trainloader = DataLoader(train_data, batch_size=1, shuffle=False, num_workers=MULTI_WORKERS_N)
testloader =  DataLoader(val_data, batch_size=1, shuffle=True, num_workers=MULTI_WORKERS_N)


### Attack Model

Create Attack model training dataset

In [5]:
attack_train_loader = create_post_train_loader(testloader, trainloader, resnet_shadow, batch_size=64, \
    multi_n= MULTI_WORKERS_N, device=DEVICE, save_path=ATT_TRAIN_DATA_PATH)

Attack Model

In [ ]:
class AttackNN(nn.Module):
    def __init__(self):
        super(AttackNN, self).__init__()
        self.fc1 = nn.Linear(3, 32)
        self.bn1 = nn.BatchNorm1d(32)  # Matches the output of fc1
        self.fc2 = nn.Linear(32, 64)
        self.bn2 = nn.BatchNorm1d(64)  # Matches the output of fc2
        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)  # Matches the output of fc3
        self.fc4 = nn.Linear(32, 16)
        self.bn4 = nn.BatchNorm1d(16)  # Matches the output of fc4
        self.fc5 = nn.Linear(16, 8)
        self.bn5 = nn.BatchNorm1d(8)   # Matches the output of fc5
        self.fc6 = nn.Linear(8, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = torch.relu(self.bn2(self.fc2(x)))
        x = torch.relu(self.bn3(self.fc3(x)))
        x = torch.relu(self.bn4(self.fc4(x)))
        x = self.dropout(torch.relu(self.bn5(self.fc5(x))))
        x = self.sigmoid(self.fc6(x))
        return x


Train Attack model

In [ ]:
attack_model = AttackNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model.parameters(), lr=LEARNING_R)

train_or_load(attack_model, attack_train_loader, optimizer, criterion, EPOCHS, ATTACK_MODEL_PATH)


Epoch: 1
tensor([[8.2618, 0.9000, 0.6849]])
tensor([1.])
Batch 0 with Loss: 0.9331651329994202
tensor([[12.1451, -0.6677, -4.9435]])
tensor([1.])
Batch 50 with Loss: 0.8166717290878296
tensor([[16.4935,  1.0791, -2.1723]])
tensor([0.])
Batch 100 with Loss: 0.6835198402404785
tensor([[10.6089,  1.8982, -0.6269]])
tensor([1.])
Batch 150 with Loss: 0.7200303077697754
tensor([[10.8100,  1.0248,  0.4482]])
tensor([0.])
Batch 200 with Loss: 0.7252645492553711
tensor([[11.3257,  3.8776, -0.8593]])
tensor([1.])
Batch 250 with Loss: 0.7747058868408203
tensor([[ 7.6587,  4.8065, -0.6778]])
tensor([1.])
Batch 300 with Loss: 0.7950496673583984
tensor([[10.3120,  3.9053,  0.2060]])
tensor([1.])
Batch 350 with Loss: 0.6938363909721375
tensor([[12.0007,  0.7907, -0.0312]])
tensor([0.])
Batch 400 with Loss: 0.6996859312057495
tensor([[15.1302,  0.4700, -1.5154]])
tensor([1.])
Batch 450 with Loss: 0.6219495534896851
Epoch: 2
tensor([[ 4.8903,  3.0597, -0.0720]])
tensor([0.])
Batch 0 with Loss: 0.721537

### Evaluation

Evaluate Dataset

In [ ]:
with open(EVALUATE_DATA_PATH, "rb") as eval_f:
    eval_dataset = pickle.load(eval_f)
    
eval_dl = DataLoader(eval_dataset, batch_size=1 , shuffle=False, num_workers=MULTI_WORKERS_N)

Load Target Model

In [ ]:
# Load target model
resnet_target_model = resnet34(weights=None,num_classes=DatasetClassN.Cifar.value)
resnet_target_model.load_state_dict (torch.load (TARGET_MODEL_PATH, map_location=DEVICE)["net"])
resnet_target_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Posteriors + Member Label

In [ ]:
eval_dl_post = create_eval_post_loader (resnet_target_model, eval_dl, MULTI_WORKERS_N, DEVICE)

TypeError: create_eval_post_loader() takes 3 positional arguments but 4 were given

Evaluate

In [ ]:
evaluate_attack_model(attack_model, optimizer, eval_dl_post)